# Styling Tool for SAR in Africa  

* **Products used:** 
[s1_rtc]()


## Background

The [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1) mission is composed of a constellation of two Synthetic Aperture Radar (SAR) satellites, Sentinel-1A and Sentinel-1B, sharing the same orbital plane. Sentinel-1 provides dual polarisation capability, very short revisit times, and rapid product delivery. The mission currently collects data every 12 days over Africa at a spatial resolution of approximately 20m. Sentinel-1A was launched on 3 April 2014 and Sentinel-1B followed on 25 April 2016. Two other spacecraft (Sentinel-1C and Sentinel-1D) are planned to replace the first two satellites at the end of their operational lifespan. For more information on the Sentinel-1 platforms and applications, check out the [European Space Agency](https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-1) website.

SAR data has the advantage of operating at wavelengths not impeded by cloud cover and can acquire data over a site during the day or night. The Sentinel-1 mission is the European Radar Observatory for the Copernicus joint initiative of the European Commission (EC) and the European Space Agency (ESA) that can offer reliable and repeated wide area monitoring with its SAR instrument.

Radar backscatter measures the amount of microwave radiation reflected back to the centre from the ground surface. This measurement is sensitive to surface roughness, moisture content, and viewing geometry. DEAfrica provides Sentinel-1 backscatter as Radiometric Terrain Corrected (RTC) gamma-0 (γ0) where variation due to changing observation geometries has been mitigated.

DE Africa provides Sentinel-1 data acquired in Interferometric Wide Swath (IW) mode and with dual polarisation (VV and VH). The dual polarisation backscatter timeseries can be used in applications for forests, agriculture, wetlands, and land cover classification. SAR's ability to see through clouds makes it critical for mapping and monitoring land cover changes in the wet tropics. 

## Description

In this notebook we will load sentinel-1 Radiometric Terrain Corrected (RTC) SAR backscatter data using `dc.load()` to return a time series of satellite images. The returned xarray.Dataset dataset will contain analysis ready images for the whole continent. 
Then we will focus on specific regions in the continent (mainly in the brightest and darkest areas) and run a few tests, to see whether the continental-scale styling needs extra stretching. 

Topics covered include:
1. Inspecting the Sentinel-1 product and measurements available in the datacube
2. Using the native `dc.load()` function to load in Sentinel-1 data
3. Using OWS styling scheme for setting the continental style 
4. Query on a few test sites to check the styling in small scale 
***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Load key Python packages and supporting functions for the analysis.

In [1]:
%matplotlib inline

import os
import datacube
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt
import glob 
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show

import deafrica_tools.temporal as ts
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.plotting import display_map, rgb
from datacube.utils.aws import configure_s3_access
from deafrica_tools.classification import HiddenPrints
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from deafrica_tools.dask import create_local_dask_cluster
configure_s3_access(aws_unsigned=True, cloud_defaults=True)

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:38721 Dashboard: /user/neginm/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Analysis parameters
The following cell sets important parameters for the analysis:
* `product`: The satellite product to load. Either Sentinel-2: `'s2_l2a'`, or Sentinel-1: `'s1_rtc'`
* `lat`: The central latitude to analyse (e.g. `-10.6996`).
* `lon`: The central longitude to analyse (e.g. `35.2708`).
* `time_range`: The year range to analyse (e.g. `('2017-01-01', '2019-12-30')`).
* `resolution`: The pixel resolution, in metres, of the returned dataset

In [3]:
product = 's1_rtc'
#set the location 
# Whole Africa 
lat_range = (-34.97,35.89)
lon_range = (-17.55, 50.53)
# Set the range of dates for the analysis
time_range = ('2020-01-01','2020-01-12')
min_gooddata = 0.15
resolution = (-5000,5000)


### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [4]:
dc = datacube.Datacube(app='SARTool')

In [5]:
dc.list_products().loc[dc.list_products()['description'].str.contains('radar')]

,name,description,platform,product_family,lat,lon,creation_time,region_code,dataset_maturity,time,instrument,product_type,label,format,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,,,,
3,s1_rtc,Sentinel 1 Gamma0 normalised radar backscatter,None,NaN,None,None,None,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN


## Load Sentinel-1 radar backscatter data and cloud-masked Sentinel-2 

The first step is to load Sentinel-1 radar backscatter data for the specified area of interest and time range. 
`dc.load()` function is used to return a time series of satellite images. The returned xarray.Dataset dataset will contain analysis ready images over agriculture field in part of Tanzania.

In [6]:
# Create a reusable query for Sentinel-1
query_S1 = {
    'x': lon_range,
    'y': lat_range,
    'time': time_range,
    'measurements': ['vv', 'vh', 'mask','angle'],
    'resolution': resolution,
    'output_crs': 'epsg:6933',
    'group_by':'solar_day'
}

# # Load available data from Sentinel-1
S1 = dc.load(product=product,
             dask_chunks={'x': 1000, 'y': 1000},
                **query_S1)
S1['vhvv'] = S1.vh.where(S1.mask==1) / S1.vv.where(S1.mask==1)

In [7]:
#Extract monthly sample and then caluclate the mean with response to time 
avg_S1= S1.resample(time='MS').mean(dim='time').compute()

In [8]:
avg_S1

<xarray.Dataset>
Dimensions:      (time: 1, x: 1314, y: 1699)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-01
  * y            (y) float64 4.292e+06 4.288e+06 ... -4.192e+06 -4.198e+06
  * x            (x) float64 -1.692e+06 -1.688e+06 ... 4.868e+06 4.872e+06
    spatial_ref  int32 6933
Data variables:
    vv           (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    vh           (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    mask         (time, y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    angle        (time, y, x) float64 255.0 255.0 255.0 ... 255.0 255.0 255.0
    vhvv         (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan

In [9]:
# We keep the min values for each polarisation band with zero percentile which is very close to zero. In order to have exposure for the two percentile to low values, we set 
# the max values to 98 percentile. 
max_vv = avg_S1.vv.quantile(0.99)
max_vh = avg_S1.vh.quantile(0.99)
max_vhvv = avg_S1.vhvv.quantile(0.99)

In [10]:
print(max_vv,max_vh,max_vhvv)

<xarray.DataArray 'vv' ()>
array(0.28221259)
Coordinates:
    quantile  float64 0.99 <xarray.DataArray 'vh' ()>
array(0.06091451)
Coordinates:
    quantile  float64 0.99 <xarray.DataArray 'vhvv' ()>
array(0.48878537)
Coordinates:
    quantile  float64 0.99


In [11]:
rgb_cfg= {
    "name": "vv_vh_vv_over_vv",
    "title": "VV, VH and VH/VV",
    "abstract": "False colour representation ofVV, VH and VH/VV for R, G and B respectively",
    "additional_bands": [],
    "components": {
        "red": {"vv":1.0,"scale_range":[0.0,0.28]},
        "green": {"vh":1.0,"scale_range":[0.0,0.06]},
        "blue": {
            "function": "datacube_ows.band_utils.band_quotient",
            "mapped_bands": True,
            "kwargs": {"band1": "vh", "band2": "vv", "scale_from":[0.0,0.49]},
        },
    },
}

In [12]:
from datacube_ows.styles.api import StandaloneStyle, apply_ows_style_cfg, xarray_image_as_png
xr_image = apply_ows_style_cfg(rgb_cfg, avg_S1)
png_image = xarray_image_as_png(xr_image)
with open("../Real_world_examples/Styled_99/Continent.png","wb") as fp:
    fp.write(png_image)

### Test Sites 

Loading Sentinel-1 dataset for different test sites and considering the fact that resolution should be in full resolution.  

In [90]:
#Timeframe and Resolution settings:
time_range = ('2020-01-01','2020-01-12')
min_gooddata = 0.15
resolution = (-20,20)

#Test Area A:Desert in Mali
lat_A = (16.10,16.32)
lon_A= (-3.80,-4.10)

#Test Area B: Forests in Congo 
lat_B = (-3.33,-3.13)
lon_B = (16.52,16.67)

#Test Area C: Etosha Pan-Namibia
lat_C = (-18.5,-18.7)
lon_C = (15.45,16.00)

#Test Area D: 
#Around Cairo
# lat_D = (30.13,30.23)
# lon_D = (30.43,30.66)
#Downtown Cairo
lat_D = (30.03,30.07)
lon_D = (31.19,31.25)

#Test Area E: Lake ihorty Madagascar
lat_E = (-21.73,-22.07)
lon_E = (43.46,43.95)

In [91]:
# Create a reusable query for Sentinel-1
query_A = {
    'x': lon_A,
    'y': lat_A,
    'time': time_range,
    'measurements': ['vv', 'vh', 'mask','angle'],
    'resolution': resolution,
    'output_crs': 'epsg:6933',
    'group_by':'solar_day'
}

# # Load available data from Sentinel-1
S1A = dc.load(product='s1_rtc',
             dask_chunks={'x': 1000, 'y': 1000},
                **query_A)
S1A['vhvv'] = S1A.vh.where(S1A.mask==1) / S1A.vv.where(S1A.mask==1)

In [92]:
#Extract monthly sample and then caluclate the mean with response to time 
S1A_avg= S1A.resample(time='MS').mean(dim='time').compute()

In [93]:
#use the 0.99 percentile 
max_vv = S1A_avg.vv.quantile(0.99)
max_vh = S1A_avg.vh.quantile(0.99)
max_vhvv = S1A_avg.vhvv.quantile(0.99)

In [94]:
print(max_vv,max_vh,max_vhvv)

<xarray.DataArray 'vv' ()>
array(0.30363022)
Coordinates:
    quantile  float64 0.99 <xarray.DataArray 'vh' ()>
array(0.05869113)
Coordinates:
    quantile  float64 0.99 <xarray.DataArray 'vhvv' ()>
array(0.6738258)
Coordinates:
    quantile  float64 0.99


In [101]:
pure_vv_cfg = {
    "components": {
        "red": {
         "vv": 1.0,"scale_range":[0.0,0.28]
        },
        "green": {
         "vv": 1.0,"scale_range":[0.0,0.28]
        },
        "blue": {
         "vv": 1.0,"scale_range":[0.0,0.28]
        },
    },
}

In [12]:
pure_vh_cfg = {
    "components": {
        "red": {
         "vh": 1.0,"scale_range":[0.0,0.06]
        },
        "green": {
         "vh": 1.0,"scale_range":[0.0,0.06]
        },
        "blue": {
         "vh": 1.0,"scale_range":[0.0,0.06]
        },
    },
}

In [87]:
rgb_cfg_A= {
    "name": "vv_vh_vv_over_vv",
    "title": "VV, VH and VH/VV",
    "abstract": "False colour representation ofVV, VH and VH/VV for R, G and B respectively",
    "additional_bands": [],
    "components": {
        "red": {"vv":1.0,"scale_range":[0.0,0.30]},
        "green": {"vh":1.0,"scale_range":[0.0,0.06]},
        "blue": {
            "function": "datacube_ows.band_utils.band_quotient",
            "mapped_bands": True,
            "kwargs": {"band1": "vh", "band2": "vv", "scale_from":[0.0,0.67]},
        },
    },
}

In [13]:
from datacube_ows.styles.api import StandaloneStyle, apply_ows_style_cfg, xarray_image_as_png
xr_image_A = apply_ows_style_cfg(pure_vh_cfg, avg_S1)
png_image = xarray_image_as_png(xr_image_A)
with open("../Real_world_examples/Styled_99/Continent_VH.png","wb") as fp:
    fp.write(png_image)

In [56]:
#documenting the values 

#Desert area:
# vv = 0.30
# vh = 0.06
# vhvv = 0.67

#Forest area: 
# vv = 0.46
# vh = 0.094
# vhvv = 0.51

# vv = 0.5
# vh = 0.1
# vhvv = 0.58

#Salt Lake area 
# vv = 0.15
# vh = 0.04
# vhvv = 0.59

# vv = 0.16
# vh = 0.04
# vhvv = 0.73

#Cairo area: 
#vv = 0.26
#vh = 0.044
#vhvv = 0.368

# vv= 0.26
# vh= 0.05
# vhvv= 0.41

#Cairo CBD area:
# vv= 0.65
# vh= 0.09
# vhvv= 0.51

#area in madagascar
# vv= 0.25
# vh= 0.06
# vhvv= 0.55

### Show the PNG 

The following steps will be used for comparing the style in different regions. 

In [95]:
#setting a search criteria for all tiff files that we want to use 
dirpath = '..//Real_world_examples/Styled_99/TestA'
#Search criteria
search_criteria = "*.png"
q = os.path.join(dirpath,search_criteria)
print(q)

..//Real_world_examples/Styled_99/TestA/*.png


In [96]:
#listing all the files with searching criteria 
files = glob.glob(q)
files

['..//Real_world_examples/Styled_99/TestA/TestSiteA_Desert_localStretch.png',
 '..//Real_world_examples/Styled_99/TestA/TestSiteA_Desert.png']

In [97]:
#create an empty list for all datafiles that is going to be part of mosaic 
images = []

In [98]:
for fp in files:
    src = rasterio.open(fp)
    images.append(src)
images

[<open DatasetReader name='..//Real_world_examples/Styled_99/TestA/TestSiteA_Desert_localStretch.png' mode='r'>,
 <open DatasetReader name='..//Real_world_examples/Styled_99/TestA/TestSiteA_Desert.png' mode='r'>]

In [100]:
#create four plots next to each other 
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, figsize=(12, 4))


# Plot first four files
show(images[0], ax=ax1, title='vhvv_nochange')
show(images[1], ax=ax2, title='vvvh_nochange')
show(images[2], ax =ax3, title='vhvv_change')

# # Do not show y-ticks values in last three axis
# for ax in [ax1]:
#     ax.yaxis.set_visible(False)

<AxesSubplot:title={'center':'vvvh_nochange'}>

## Conclusions

In the example above, we can see these four fields are following the same cropping schedule and are therefore likely the same species of crop. We can also observe intra-field differences in the rates of growth, and in the NDVI values at different times of the season, which may be attributable to differences in soil quality, watering intensity, or other farming practices. 

Phenology statistics are a powerful way to summarise the seasonal cycle of a plant's life.  Per-pixel plots of phenology can help us understand the timing of vegetation growth and seasonality across large areas and across diverse plant species as every pixel is treated as an independent series of observations. This could be important, for example, if we wanted to assess how the growing seasons are shifting as the climate warms.  


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [36]:
print(datacube.__version__)

1.8.4.dev63+g6ee0462c


**Last Tested:**

In [37]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2021-05-06'